In [3]:
from nmf import *
from utils import *
import numpy as np
from projection import project_onto_Sigma
from data_generation import generate_separable_data
from data_generation import generate_separable_data_with_noise

In [4]:
# Main algorithm: Kernel Separable NMF
# ToDo: integrate in NMF class in nmf.py
import numpy as np

def mean_groups_diff(X):
    """
    Compute the difference between the means of the two groups of diagonal norms
    of the matrix X, where the groups are defined by a gap in the sorted diagonal norms.
    
    Parameters:
        X : np.ndarray
            Input matrix (typically the output of a kernel NMF).
    
    Returns:
        diff : float
            The difference between the means of the two groups.
    """
    # Sort the diagonal elements in descending order
    diag_X = np.diag(X)
    sorted_diag = np.sort(diag_X)[::-1]  # Descending order

    # Compute consecutive differences
    diffs = sorted_diag[:-1] - sorted_diag[1:]

    # Find the index of the largest gap
    gap_index = np.argmax(diffs)

    # Split into two groups based on the gap index
    large_group = sorted_diag[:gap_index+1]
    small_group = sorted_diag[gap_index+1:]

    # Calculate means and their difference
    mean_large = np.mean(large_group)
    mean_small = np.mean(small_group)

    return mean_large - mean_small

def group_gap(X):
    """
    Compute the gap between the two groups of diagonal norms of the matrix X,
    where the groups are defined by a gap in the sorted diagonal norms.
    Parameters:
        X : np.ndarray
            Input matrix (typically the output of a kernel NMF).
    Returns:    
        gap : float
            The gap between the means of the two groups.
            """
    # Sort the diagonal elements in descending order
    diag_X = np.diag(X)
    sorted_diag = np.sort(diag_X)[::-1]  # Descending order

    # Compute consecutive differences
    diffs = sorted_diag[:-1] - sorted_diag[1:]

    # Find the index of the largest gap
    gap = np.max(diffs)

    return gap

def kernel_separable_nmf(K, lambda_param=1.0, max_iter=100, r=3, verbose=False, tol=1e-3, gap_tolerance=None):
    """
    Kernel Separable NMF algorithm with convergence tolerance.

    Parameters:
        K            : Kernel matrix (typically M^T M).
        lambda_param : Regularization parameter.
        max_iter     : Maximum number of iterations.
        r            : Number of columns (indices) to select (expected anchors).
        verbose      : If True, prints progress.
        tol          : Convergence threshold for diagonal updates.

    Returns:
        X         : The final matrix X after updates.
        indices   : The indices corresponding to the r largest diagonal entries of X.
        diag_norms: List of diagonal vectors (one per iteration) for analysis.
    """
    n = K.shape[0]
    X = np.random.rand(n, n)
    I = np.eye(n)
    epsilon = 1e-10
    diag_norms = []
    mean_groups_diffs = []  # To store the mean differences for analysis

    # For checking convergence
    prev_diag = None  # Will store the diagonal of X at the previous iteration
    
    for it in range(max_iter):
        KX = K @ X
        denom = 2 * lambda_param * KX + epsilon
        numerator = 2 * lambda_param * K - I
        
        update_factor = numerator / denom
        X = X * update_factor
        
        #print(1, np.trace(X)+lambda_param*np.linalg.norm(M-M@X))
        X = project_onto_Sigma(X)  
        #print(2, np.trace(X)+lambda_param*np.linalg.norm(M-M@X))
        X_diagonal = np.diag(X)
        diag_norms.append(X_diagonal)
        mean_groups_diffs.append(mean_groups_diff(X))
        # Convergence check: once we have a previous diagonal, compare norms
        if prev_diag is not None:
            diff = np.linalg.norm(X_diagonal - prev_diag)
            if diff < tol:
                if verbose:
                    print(f"Converged at iteration {it} with diagonal diff={diff:.6e}")
                break
        
        if gap_tolerance is not None:
            gap = group_gap(X)
            if gap > gap_tolerance:
                if verbose:
                    print(f"Converged at iteration {it} with gap={gap:.6e}")
                break
        
        prev_diag = X_diagonal
        
        # Optional progress logging
        if verbose and (it % 10 == 0 or it == max_iter - 1):
            diag_norm = np.linalg.norm(X_diagonal)
            print(f"Iteration {it}: ||diag(X)|| = {diag_norm:.4f}")
            #print(f"iteration {it}: {np.trace(X)+lambda_param*np.linalg.norm(M-M@X):.4f} (objective value)")
    
    # After stopping (due to convergence or hitting max_iter),
    # pick the top r diagonal entries as "selected variables"
    final_diag = np.diag(X)  # or X_diagonal if you trust the break didn't skip an update
    indices = np.argsort(final_diag)[-r:][::-1]
    
    return X, indices, diag_norms, mean_groups_diffs

In [14]:
# ----- Example of usage -----
lambda_param = 50 # Higher value for lower sparsity on the diagonal of X
max_iter = 1000
n_anchors = 3

#K = generate_separable_data(n_anchors=n_anchors, n_samples=100, dimension=70, seed=2, kernel=None, sparsity_on_H=0)
K = generate_separable_data_with_noise(n_anchors=n_anchors, n_samples=100, dimension=70, seed=2, kernel=None, sparsity_on_H=0, sigma=0.1)
X_kernel, selected_indices, diag_norms, mean_groups_diffs = kernel_separable_nmf(K, lambda_param, max_iter, n_anchors, verbose=True, gap_tolerance=0.4)

# ----- Print results -----
print("\nKernel matrix K:")
print(K)
print("\nFinal X matrix from Kernel Separable NMF:")
print(X_kernel)
print("\nIndices selected for factorization (should include the anchors):")
print(selected_indices)

Iteration 0: ||diag(X)|| = 0.0949
Iteration 10: ||diag(X)|| = 0.3748
Iteration 20: ||diag(X)|| = 0.7888
Iteration 30: ||diag(X)|| = 1.2303
Iteration 40: ||diag(X)|| = 1.6211
Iteration 50: ||diag(X)|| = 1.9923
Iteration 60: ||diag(X)|| = 2.3725
Iteration 70: ||diag(X)|| = 2.7715
Iteration 80: ||diag(X)|| = 3.1855
Iteration 90: ||diag(X)|| = 3.6047
Iteration 100: ||diag(X)|| = 4.0192
Iteration 110: ||diag(X)|| = 4.4204
Iteration 120: ||diag(X)|| = 4.8023
Iteration 130: ||diag(X)|| = 5.1611
Iteration 140: ||diag(X)|| = 5.4947
Iteration 150: ||diag(X)|| = 5.8022
Iteration 160: ||diag(X)|| = 6.0841
Iteration 170: ||diag(X)|| = 6.3414
Iteration 180: ||diag(X)|| = 6.5756
Iteration 190: ||diag(X)|| = 6.7886
Iteration 200: ||diag(X)|| = 6.9822
Iteration 210: ||diag(X)|| = 7.1585
Iteration 220: ||diag(X)|| = 7.3190
Iteration 230: ||diag(X)|| = 7.4656
Iteration 240: ||diag(X)|| = 7.5996
Iteration 250: ||diag(X)|| = 7.7224
Iteration 260: ||diag(X)|| = 7.8351
Iteration 270: ||diag(X)|| = 7.9386
Ite

In [17]:
# ----- Example of usage -----
lambda_param = 50 # Higher value for lower sparsity on the diagonal of X
max_iter = 1000
n_anchors = 3

#K = generate_separable_data(n_anchors=n_anchors, n_samples=100, dimension=70, seed=2, kernel=None, sparsity_on_H=0)
K = generate_separable_data_with_noise(n_anchors=n_anchors, n_samples=100, dimension=70, seed=2, kernel="Gaussian", sparsity_on_H=0, sigma=0.01)
X_kernel, selected_indices, diag_norms, mean_groups_diffs = kernel_separable_nmf(K, lambda_param, max_iter, n_anchors, verbose=True, gap_tolerance=0.4)

# ----- Print results -----
print("\nKernel matrix K:")
print(K)
print("\nFinal X matrix from Kernel Separable NMF:")
print(X_kernel)
print("\nIndices selected for factorization (should include the anchors):")
print(selected_indices)

Iteration 0: ||diag(X)|| = 0.0898
Iteration 10: ||diag(X)|| = 0.2529
Iteration 20: ||diag(X)|| = 0.4679
Iteration 30: ||diag(X)|| = 0.7014
Iteration 40: ||diag(X)|| = 0.8799
Iteration 50: ||diag(X)|| = 1.0067
Iteration 60: ||diag(X)|| = 1.0984
Iteration 70: ||diag(X)|| = 1.1666
Iteration 80: ||diag(X)|| = 1.2183
Iteration 90: ||diag(X)|| = 1.2585
Iteration 100: ||diag(X)|| = 1.2904
Iteration 110: ||diag(X)|| = 1.3153
Iteration 120: ||diag(X)|| = 1.3351
Iteration 130: ||diag(X)|| = 1.3506
Iteration 140: ||diag(X)|| = 1.3631
Iteration 150: ||diag(X)|| = 1.3731
Iteration 160: ||diag(X)|| = 1.3825
Iteration 170: ||diag(X)|| = 1.3904
Iteration 180: ||diag(X)|| = 1.3970
Iteration 190: ||diag(X)|| = 1.4021
Iteration 200: ||diag(X)|| = 1.4057
Iteration 210: ||diag(X)|| = 1.4086
Iteration 220: ||diag(X)|| = 1.4109
Converged at iteration 228 with gap=4.002130e-01

Kernel matrix K:
[[1.         0.76090212 0.75238695 ... 0.96284599 0.95449715 0.86570549]
 [0.76090212 1.         0.76158229 ... 0.81